# Basic Function

In [ ]:
from math import comb
from itertools import combinations, product
import pandas as pd
import time
import sympy


In [ ]:
# calculating the run list of (0)strx(1)
def runstring(strx):
    x = "0"+strx+"1"
    r_list = [0]
    rank = 0
    for i in range(1, len(x)):
        if x[i] != x[i-1]:
            rank += 1
        r_list.append(rank)
    return r_list

# f1r function
def f1rx(strx):
    return sum(runstring(strx))

# f2r function
def f2rx(strx):
    rstring = runstring(strx)
    tot = 0
    for i in rstring:
        tot += comb(i,2)
    return tot

# Calculate number of new runs of x, a 2-insertion superstring on y
def count_new_runs_x_y(x, y):
    r_x = runstring(x)
    r_y = runstring(y)
    return r_x[-1] - r_y[-1]

# function to find prime p (n+2, 2n]
def find_prime(n):
    for p in sympy.primerange(n + 3, 2 * n):
        return p
    return None

In [ ]:
# testing:

s = "001000111010"
print(f1(s), f2(s))
# f1 = 3+7+8+9+11 = 38
# f2 = (3 c 2)+(7 c 2)+(8 c 2)+(9 c 2)+(11 c 2) = 143

In [ ]:
# Given n, create a list of y's of all the strings of length n-2

#  Input: n-2 value
# Output: a list of all length n-2 strings
def generate_binary_strings(n_minus_2):
    return [''.join(seq) for seq in product('01', repeat=n_minus_2)]

In [ ]:
# Insert Characters to Form Length n strings

#  Input: string y
# Output: list of all possible 2 insertion on y
def generate_2_insertions(y, char='01'):
    n = len(y) + 2
    inserted_strings = set()  # Use a set to avoid duplicates
    
    for i in range(n):
        for j in range(i + 1, n):
            for c1 in char:
                for c2 in char:
                    inserted_str = y[:i] + c1 + y[i:j-1] + c2 + y[j-1:]
                    inserted_strings.add(inserted_str)
                    
    return list(inserted_strings)

In [ ]:
# check if anything new duplicates created.

# Method 3: Using (f1r mod p, f2r mod p, #0, #1, #01, #10)

In [ ]:
import pandas as pd
import time
import sympy

def find_prime_one_half_n(n):
    for p in sympy.primerange(int(1.5 * n + 1), 2 * n):
        return p
    return None

def test_2_deletion_f1f2modp_num0_1_01_10_THREE(n):
    # Find the prime p such that 1.5n < p < 2n
    p = find_prime_one_half_n(n)
    if p is None:
        raise ValueError("No prime found in the range (1.5n, 2n).")
    
    start_time = time.time()  # Start the timer
    
    # Generate all possible binary strings of length n-2
    y_s = generate_binary_strings(n - 2)
    max_consecutive_duplicates_global = 0
    sequence_with_max_duplicates = []  # To store the sequences of consecutive terms with maximum duplicates

    # For each y in y_s, generate all possible x' of length n by 2 insertions
    for y in y_s:
        x_primes = generate_2_insertions(y)
        
        results = []
        for x_prime in x_primes:
            f1r_value = f1rx(x_prime) % p  # Calculate f1r mod p
            f2r_value = f2rx(x_prime) % p  # Calculate f2r mod p
            new_runs = count_new_runs_x_y(x_prime, y) % p  # Calculate new_runs mod p

            # Counting number of 0, 1, 01, 10
            num_0_val = x_prime.count("0")
            num_1_val = x_prime.count("1")
            num_01_val = x_prime.count("01")
            num_10_val = x_prime.count("10")
            
            results.append({
                'y': y,
                'x_prime': x_prime,
                'f1r': f1r_value,
                'f2r': f2r_value,
                'new_runs': new_runs,
                'num_0': num_0_val,
                'num_1': num_1_val,
                'num_01': num_01_val,
                'num_10': num_10_val
            })
        
        # Create DataFrame for the current y's results
        df_y = pd.DataFrame(results)
        
        # Sort the DataFrame by the tuple (f1r mod p, f2r mod p, num_0, num_1, num_01, num_10, new_runs)
        df_y_sorted = df_y.sort_values(
            by=['f1r', 'f2r', 'num_0', 'num_1', 'num_01', 'num_10', 'new_runs'],
            ascending=[True, True, True, True, True, True, True]
        )
        
        # Check for consecutive duplicates and track sequences with max duplicates
        current_count = 1
        current_consecutive_terms = [df_y_sorted.iloc[0]]  # Initialize with the first term
        
        for i in range(1, len(df_y_sorted)):
            if (
                df_y_sorted.iloc[i]['f1r'] == df_y_sorted.iloc[i-1]['f1r'] and
                df_y_sorted.iloc[i]['f2r'] == df_y_sorted.iloc[i-1]['f2r'] and
                df_y_sorted.iloc[i]['new_runs'] == df_y_sorted.iloc[i-1]['new_runs'] and
                df_y_sorted.iloc[i]['num_0'] == df_y_sorted.iloc[i-1]['num_0'] and
                df_y_sorted.iloc[i]['num_1'] == df_y_sorted.iloc[i-1]['num_1'] and
                df_y_sorted.iloc[i]['num_01'] == df_y_sorted.iloc[i-1]['num_01'] and
                df_y_sorted.iloc[i]['num_10'] == df_y_sorted.iloc[i-1]['num_10']
            ):
                current_count += 1
                current_consecutive_terms.append(df_y_sorted.iloc[i])
            else:
                if current_count > max_consecutive_duplicates_global:
                    max_consecutive_duplicates_global = current_count
                    sequence_with_max_duplicates = [
                        [
                            current_consecutive_terms[0]['y'],
                            [term['x_prime'] for term in current_consecutive_terms],
                            current_consecutive_terms[0]['f1r'],
                            current_consecutive_terms[0]['f2r'],
                            current_consecutive_terms[0]['num_0'],
                            current_consecutive_terms[0]['num_1'],
                            current_consecutive_terms[0]['num_01'],
                            current_consecutive_terms[0]['num_10'],
                            current_consecutive_terms[0]['new_runs']
                        ]
                    ]
                elif current_count == max_consecutive_duplicates_global:
                    sequence_with_max_duplicates.append(
                        [
                            current_consecutive_terms[0]['y'],
                            [term['x_prime'] for term in current_consecutive_terms],
                            current_consecutive_terms[0]['f1r'],
                            current_consecutive_terms[0]['f2r'],
                            current_consecutive_terms[0]['num_0'],
                            current_consecutive_terms[0]['num_1'],
                            current_consecutive_terms[0]['num_01'],
                            current_consecutive_terms[0]['num_10'],
                            current_consecutive_terms[0]['new_runs']
                        ]
                    )
                # Reset for the next sequence
                current_count = 1
                current_consecutive_terms = [df_y_sorted.iloc[i]]
        
        # Final check after loop
        if current_count > max_consecutive_duplicates_global:
            max_consecutive_duplicates_global = current_count
            sequence_with_max_duplicates = [
                [
                    current_consecutive_terms[0]['y'],
                    [term['x_prime'] for term in current_consecutive_terms],
                    current_consecutive_terms[0]['f1r'],
                    current_consecutive_terms[0]['f2r'],
                    current_consecutive_terms[0]['num_0'],
                    current_consecutive_terms[0]['num_1'],
                    current_consecutive_terms[0]['num_01'],
                    current_consecutive_terms[0]['num_10'],
                    current_consecutive_terms[0]['new_runs']
                ]
            ]
        elif current_count == max_consecutive_duplicates_global:
            sequence_with_max_duplicates.append(
                [
                    current_consecutive_terms[0]['y'],
                    [term['x_prime'] for term in current_consecutive_terms],
                    current_consecutive_terms[0]['f1r'],
                    current_consecutive_terms[0]['f2r'],
                    current_consecutive_terms[0]['num_0'],
                    current_consecutive_terms[0]['num_1'],
                    current_consecutive_terms[0]['num_01'],
                    current_consecutive_terms[0]['num_10'],
                    current_consecutive_terms[0]['new_runs']
                ]
            )
    
    end_time = time.time()  # End the timer
    runtime = end_time - start_time  # Calculate the runtime
    print(f"Time: {runtime:.2f} seconds")

    print(f"For n={n}, Maximum consecutive duplicates among all y's: {max_consecutive_duplicates_global}\n")
    for seq in sequence_with_max_duplicates:
        print(f"y = {seq[0]},  2-Insertion: {seq[1]},  f1r mod {p} = {seq[2]}, f2r mod {p} = {seq[3]}, NewRuns = {seq[8]}, #0 = {seq[4]}, #1 = {seq[5]}, #01 = {seq[6]}, #10 = {seq[7]}, ")
        
    # Return the maximum number of consecutive duplicates and the sequences that achieve this max
    return max_consecutive_duplicates_global, sequence_with_max_duplicates


# result

In [ ]:
THREE_L_10, THREE_seq_10 = test_2_deletion_f1f2modp_num0_1_01_10_THREE(10)

In [ ]:
THREE_L_11, THREE_seq_11 = test_2_deletion_f1f2modp_num0_1_01_10_THREE(11)

In [ ]:
THREE_L_12, THREE_seq_12 = test_2_deletion_f1f2modp_num0_1_01_10_THREE(12)

In [ ]:
THREE_L_13, THREE_seq_13 = test_2_deletion_f1f2modp_num0_1_01_10_THREE(13)

In [ ]:
THREE_L_14, THREE_seq_14 = test_2_deletion_f1f2modp_num0_1_01_10_THREE(14)

In [ ]:
THREE_L_15, THREE_seq_15 = test_2_deletion_f1f2modp_num0_1_01_10_THREE(15)

In [ ]:
THREE_L_16, THREE_seq_16 = test_2_deletion_f1f2modp_num0_1_01_10_THREE(16)

In [ ]:
THREE_L_17, THREE_seq_17 = test_2_deletion_f1f2modp_num0_1_01_10_THREE(17)

In [ ]:
THREE_L_18, THREE_seq_18 = test_2_deletion_f1f2modp_num0_1_01_10_THREE(18)

In [ ]:
THREE_L_19, THREE_seq_19 = test_2_deletion_f1f2modp_num0_1_01_10_THREE(19)

In [ ]:
THREE_L_20, THREE_seq_20 = test_2_deletion_f1f2modp_num0_1_01_10_THREE(20)

In [ ]:
# check the length of the list
# new collision

# find the new pair.


# Finding new duplicates

In [ ]:
# this function sort based on (f1r, f2r, #newruns, #0, #1, #01, #10), without mod p

def test_2_deletion_f1f2_num0_1_01_10_THREE(n):

    start_time = time.time()  # Start the timer
    
    # Generate all possible binary strings of length n-2
    y_s = generate_binary_strings(n - 2)
    max_consecutive_duplicates_global = 0
    sequence_with_max_duplicates = []  # To store the sequences of consecutive terms with maximum duplicates

    # For each y in y_s, generate all possible x' of length n by 2 insertions
    for y in y_s:
        x_primes = generate_2_insertions(y)
        
        results = []
        for x_prime in x_primes:
            f1r_value = f1rx(x_prime)  # Calculate f1r mod p
            f2r_value = f2rx(x_prime)  # Calculate f2r mod p
            new_runs = count_new_runs_x_y(x_prime, y)  # Calculate new_runs mod p

            # Counting number of 0, 1, 01, 10
            num_0_val = x_prime.count("0")
            num_1_val = x_prime.count("1")
            num_01_val = x_prime.count("01")
            num_10_val = x_prime.count("10")
            
            results.append({
                'y': y,
                'x_prime': x_prime,
                'f1r': f1r_value,
                'f2r': f2r_value,
                'new_runs': new_runs,
                'num_0': num_0_val,
                'num_1': num_1_val,
                'num_01': num_01_val,
                'num_10': num_10_val
            })
        
        # Create DataFrame for the current y's results
        df_y = pd.DataFrame(results)
        
        # Sort the DataFrame by the tuple (f1r mod p, f2r mod p, num_0, num_1, num_01, num_10, new_runs)
        df_y_sorted = df_y.sort_values(
            by=['f1r', 'f2r', 'num_0', 'num_1', 'num_01', 'num_10', 'new_runs'],
            ascending=[True, True, True, True, True, True, True]
        )
        
        # Check for consecutive duplicates and track sequences with max duplicates
        current_count = 1
        current_consecutive_terms = [df_y_sorted.iloc[0]]  # Initialize with the first term
        
        for i in range(1, len(df_y_sorted)):
            if (
                df_y_sorted.iloc[i]['f1r'] == df_y_sorted.iloc[i-1]['f1r'] and
                df_y_sorted.iloc[i]['f2r'] == df_y_sorted.iloc[i-1]['f2r'] and
                df_y_sorted.iloc[i]['new_runs'] == df_y_sorted.iloc[i-1]['new_runs'] and
                df_y_sorted.iloc[i]['num_0'] == df_y_sorted.iloc[i-1]['num_0'] and
                df_y_sorted.iloc[i]['num_1'] == df_y_sorted.iloc[i-1]['num_1'] and
                df_y_sorted.iloc[i]['num_01'] == df_y_sorted.iloc[i-1]['num_01'] and
                df_y_sorted.iloc[i]['num_10'] == df_y_sorted.iloc[i-1]['num_10']
            ):
                current_count += 1
                current_consecutive_terms.append(df_y_sorted.iloc[i])
            else:
                if current_count > max_consecutive_duplicates_global:
                    max_consecutive_duplicates_global = current_count
                    sequence_with_max_duplicates = [
                        [
                            current_consecutive_terms[0]['y'],
                            [term['x_prime'] for term in current_consecutive_terms],
                            current_consecutive_terms[0]['f1r'],
                            current_consecutive_terms[0]['f2r'],
                            current_consecutive_terms[0]['num_0'],
                            current_consecutive_terms[0]['num_1'],
                            current_consecutive_terms[0]['num_01'],
                            current_consecutive_terms[0]['num_10'],
                            current_consecutive_terms[0]['new_runs']
                        ]
                    ]
                elif current_count == max_consecutive_duplicates_global:
                    sequence_with_max_duplicates.append(
                        [
                            current_consecutive_terms[0]['y'],
                            [term['x_prime'] for term in current_consecutive_terms],
                            current_consecutive_terms[0]['f1r'],
                            current_consecutive_terms[0]['f2r'],
                            current_consecutive_terms[0]['num_0'],
                            current_consecutive_terms[0]['num_1'],
                            current_consecutive_terms[0]['num_01'],
                            current_consecutive_terms[0]['num_10'],
                            current_consecutive_terms[0]['new_runs']
                        ]
                    )
                # Reset for the next sequence
                current_count = 1
                current_consecutive_terms = [df_y_sorted.iloc[i]]
        
        # Final check after loop
        if current_count > max_consecutive_duplicates_global:
            max_consecutive_duplicates_global = current_count
            sequence_with_max_duplicates = [
                [
                    current_consecutive_terms[0]['y'],
                    [term['x_prime'] for term in current_consecutive_terms],
                    current_consecutive_terms[0]['f1r'],
                    current_consecutive_terms[0]['f2r'],
                    current_consecutive_terms[0]['num_0'],
                    current_consecutive_terms[0]['num_1'],
                    current_consecutive_terms[0]['num_01'],
                    current_consecutive_terms[0]['num_10'],
                    current_consecutive_terms[0]['new_runs']
                ]
            ]
        elif current_count == max_consecutive_duplicates_global:
            sequence_with_max_duplicates.append(
                [
                    current_consecutive_terms[0]['y'],
                    [term['x_prime'] for term in current_consecutive_terms],
                    current_consecutive_terms[0]['f1r'],
                    current_consecutive_terms[0]['f2r'],
                    current_consecutive_terms[0]['num_0'],
                    current_consecutive_terms[0]['num_1'],
                    current_consecutive_terms[0]['num_01'],
                    current_consecutive_terms[0]['num_10'],
                    current_consecutive_terms[0]['new_runs']
                ]
            )
    
    end_time = time.time()  # End the timer
    runtime = end_time - start_time  # Calculate the runtime
    print(f"Time: {runtime:.2f} seconds")

    print(f"For n={n}, Maximum consecutive duplicates among all y's: {max_consecutive_duplicates_global}\n")
    for seq in sequence_with_max_duplicates:
        print(f"y = {seq[0]},  2-Insertion: {seq[1]},  f1r = {seq[2]}, f2r  = {seq[3]}, NewRuns = {seq[8]}, #0 = {seq[4]}, #1 = {seq[5]}, #01 = {seq[6]}, #10 = {seq[7]}, ")
        
    # Return the maximum number of consecutive duplicates and the sequences that achieve this max
    return max_consecutive_duplicates_global, sequence_with_max_duplicates


In [ ]:
THREE_2_L_10, THREE_2_seq_10 = test_2_deletion_f1f2_num0_1_01_10_THREE(10)

In [ ]:
THREE_2_L_11, THREE_2_seq_11 = test_2_deletion_f1f2_num0_1_01_10_THREE(11)

In [ ]:
THREE_2_L_12, THREE_2_seq_12 = test_2_deletion_f1f2_num0_1_01_10_THREE(12)

In [ ]:
THREE_2_L_13, THREE_2_seq_13 = test_2_deletion_f1f2_num0_1_01_10_THREE(13)

In [ ]:
THREE_2_L_14, THREE_2_seq_14 = test_2_deletion_f1f2_num0_1_01_10_THREE(14)

In [ ]:
THREE_2_L_15, THREE_2_seq_15 = test_2_deletion_f1f2_num0_1_01_10_THREE(15)

In [ ]:
THREE_2_L_16, THREE_2_seq_16 = test_2_deletion_f1f2_num0_1_01_10_THREE(16)

In [ ]:
THREE_2_L_17, THREE_2_seq_17 = test_2_deletion_f1f2_num0_1_01_10_THREE(17)

In [ ]:
THREE_2_L_18, THREE_2_seq_18 = test_2_deletion_f1f2_num0_1_01_10_THREE(18)

In [ ]:
THREE_2_L_19, THREE_2_seq_19 = test_2_deletion_f1f2_num0_1_01_10_THREE(19)

# Checking if there's any new duplicates.

In [ ]:
# quick check on size difference
print(11, len(THREE_2_seq_11), len(THREE_seq_11))
print(12, len(THREE_2_seq_12), len(THREE_seq_12))
print(13, len(THREE_2_seq_13), len(THREE_seq_13))
print(14, len(THREE_2_seq_14), len(THREE_seq_14))
print(15, len(THREE_2_seq_15), len(THREE_seq_15))
print(16, len(THREE_2_seq_16), len(THREE_seq_16))
print(17, len(THREE_2_seq_17), len(THREE_seq_17))
print(18, len(THREE_2_seq_18), len(THREE_seq_18)) # give 178 new duplicates/collisions
print(19, len(THREE_2_seq_19), len(THREE_seq_19))

In [ ]:
# finding new duplicates when using mod p
def finding_new_dups(n, seq, seq_mod_p):
    start_time = time.time()  # Start the timer
    
    # Convert seq to a set of tuples for faster lookup
    seq_set = set((item[0], tuple(item[1])) for item in seq)
    
    new_dups = []
    for item_mod_p in seq_mod_p:
        item_mod_p_key = (item_mod_p[0], tuple(item_mod_p[1]))
        
        if item_mod_p_key not in seq_set:
            new_dups.append(item_mod_p)
    
    end_time = time.time()  # End the timer
    runtime = end_time - start_time  # Calculate the runtime
    print(f"\nFor n={n}: Time: {runtime:.2f} seconds")
    
    return new_dups


In [ ]:
# checking

seq1 = [['01', ['10', '111']], ['11', ['11', '011']], ['00', ['10', '101']]]
seq2 = [['01', ['10', '111']], ['11', ['11', '011']], ['00', ['10', '101']], ['00', ['11', '001']]]

# Run the finding_new_dups function
new_dups = finding_new_dups(2, seq1, seq2)

# Print the results
print("New duplicates found:\n", new_dups)


In [ ]:
new_dups_10 = finding_new_dups(10, THREE_2_seq_10, THREE_seq_10)
new_dups_11 = finding_new_dups(11, THREE_2_seq_11, THREE_seq_11)
new_dups_12 = finding_new_dups(12, THREE_2_seq_12, THREE_seq_12)
new_dups_13 = finding_new_dups(13, THREE_2_seq_13, THREE_seq_13)
new_dups_14 = finding_new_dups(14, THREE_2_seq_14, THREE_seq_14)
new_dups_15 = finding_new_dups(15, THREE_2_seq_15, THREE_seq_15)
new_dups_16 = finding_new_dups(16, THREE_2_seq_16, THREE_seq_16)
new_dups_17 = finding_new_dups(17, THREE_2_seq_17, THREE_seq_17)
new_dups_18 = finding_new_dups(18, THREE_2_seq_18, THREE_seq_18)
new_dups_19 = finding_new_dups(19, THREE_2_seq_19, THREE_seq_19)


In [ ]:
print("New duplicates found:\n", new_dups_10)
print("New duplicates found:\n", new_dups_11)
print("New duplicates found:\n", new_dups_12)
print("New duplicates found:\n", new_dups_13)
print("New duplicates found:\n", new_dups_14)
print("New duplicates found:\n", new_dups_15)
print("New duplicates found:\n", new_dups_16)
print("New duplicates found:\n", new_dups_17)
print("New duplicates found:\n", new_dups_18)
print("New duplicates found:\n", new_dups_19)

In [ ]:
def check_new_dup(aaa):

#aaa = ['000011110', ['00221110100', '00000101110']]
    y = aaa[0]
    x1 = aaa[1][0]
    x2 = aaa[1][1]
    p = find_prime_one_half_n(len(x1))

    if f1rx(x1) != f1rx(x2) or f2rx(x1) != f2rx(x2):
        print(f'new duplicate on {item}')
        print(f1rx(x1), f1rx(x2), f2rx(x1), f2rx(x2))
        print(f1rx(x1)%p, f1rx(x2)%p, f2rx(x1)%p, f2rx(x2)%p)
    else: print('this is not a new duplicate')

In [ ]:
print(len(new_dups_18))
print(len(THREE_seq_18) -  len(THREE_2_seq_18))
for item in new_dups_18:
    check_new_dup(item)

In [ ]:
print(len(new_dups_19))
print(len(THREE_seq_19) -  len(THREE_2_seq_19))
for item in new_dups_19:
    check_new_dup(item)

# Check the randomness in these new collisions

In [ ]:
import math

def is_non_random(s, threshold):
    # Check for consecutive sequences of '0' or '1'
    if '0' * threshold in s or '1' * threshold in s:
        return True
    
    # Check for consecutive sequences of '01' or '10'
    if '01' * threshold in s or '10' * threshold in s:
        return True
    
    return False

def check_random(seqs):
    n = len(seqs[0][1][0])
    threshold = math.ceil(math.log(n))  # Moved outside of the loop for efficiency
    num = [] # 1 for random , 0 for not random, hope to get all not random
    for seq in seqs:
        y = seq[0]
        x1 = seq[1][0]
        x2 = seq[1][1]
    
        # Check if x1 or x2 are non-random
        if not (is_non_random(x1, threshold) or is_non_random(x2, threshold)):
            print(f'random for {seq}')
            num.append(1)
        else:
            print('not random')
            num.append(0)
    return sum(num)

In [ ]:
check_random(new_dups_18)

In [ ]:
check_random(new_dups_19)

At least for now, we see all new collisions are considered as unusual cases, so we argue that we still guarantee a list size 2. Will run through more test.